# Aula 5 - Estudo de Caso - Recomendação de Produtos

Cada vez mais as pessoas estão realizando pedidos de compras online (até mesmo em função da pandemia). Algumas empresas de supermercados têm tentado desenvolver sistemas de recomendação para facilitar a interação do usuário com o site de compras. Imagine o cenário em que um usuário retorna frequentemente ao sistema e deseja ter o carrinho já montado com suas preferências usuais.

<img src="img/cart.png" alt="shopping-cart" style="width: 450px;"/>

Nesse estudo de caso, imagine que você é um engenheiro de *machine learning* com a tarefa de automatizar a recomendação de itens no carrinho de compra dos clientes do site. Você tem acesso a um *dataset* de dados históricos e um simulador construído em cima desses dados.

A metodologia utilizado nesse estudo de caso será dividida nas seguintes 4 etapas:

1. **Entendendo o simulador (caixa-preta)**: quais as características gerais do simulador? como é o espaço de observações e de ações?
2. **Escolhendo o algoritmo**: dentre os algoritmos estudados em aula, qual o mais adequado? E por quê? É necessário alterar o algoritmo de alguma forma?
3. **Análise da solução**: qualitativamente como podemos avaliar o desempenho da solução obtida?
4. **Decisões de modelagem - MDPs (caixa-branca)**: quais as principais escolhas de modelagem usadas no simulador? É possível melhorar?


### Objetivos:


- Estudar uma primeira abordagem para problemas de tomada de decisão com dados históricos
- Analisar e criticar escolhas de modelagem do ambiente
- Adaptar algoritmos clássicos de Deep RL para aplicações específicas
- Implementar soluções para espaço combinatório de de ações


### References

- [Instacart](https://www.instacart.com/)
- [3 Million Instacart Orders, Open Sourced](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2)
- [winderresearch/rl/gym-shopping-cart](https://gitlab.com/winderresearch/rl/gym-shopping-cart)

### Instalação

In [ ]:
!wget -N https://s3.eu-west-2.amazonaws.com/assets.winderresearch.com/data/instacart_online_grocery_shopping_2017_05_01.tar.gz

In [ ]:
!pip install gym-shopping-cart

### Imports

In [ ]:
from collections import deque, defaultdict
import copy
from datetime import datetime
import itertools
import multiprocessing as mp
import os.path as osp
import pathlib
from pprint import pprint
import time

import gym
import gym_shopping_cart
from gym_shopping_cart.data.parser import InstacartData
import numpy as np
import sonnet as snt
import tensorflow as tf
from tqdm.notebook import trange

from utils import logging
from utils.nn import initializers
from utils import replay
from utils import schedule
from utils import tf_utils
from utils import wrapper


tf_utils.set_tf_allow_growth() # necessário apenas se você dispõe de GPU

# Introdução ao problema de recomendação

In [ ]:
gz_file = pathlib.Path("instacart_online_grocery_shopping_2017_05_01.tar.gz")
data = InstacartData(gz_file)

In [ ]:
# produtos
print(f"# products  = {data.n_products()}")

# usuários
all_orders = data._merged_data()
all_users = all_orders["user_id"].unique()
print(f"# users     = {len(all_users)}")

# pedidos
df = data._orders()
print(f"# orders    = {len(df)}")

# compras (número de produtos vendidos no período)
df = data._prior_products()
print(f"# purchases = {len(df)}")

# 1. Entendendo o simulador (caixa-preta)

Quais as características gerais do simulador? Como é o espaço de observações e de ações?

In [ ]:
def make_shopping_cart_envs(max_products=5, user_id=None):
    data = InstacartData(gz_file, max_products=max_products)
    env_id = "ShoppingCart-v0"
    train_env = gym.make(env_id, data=data, user_id=user_id)
    eval_env = gym.make(env_id, data=data, user_id=user_id)
    test_env = wrapper.ShoppingCartWrapper(gym.make(env_id, data=data, user_id=user_id))
    if user_id:
        for env in [train_env, eval_env, test_env]:
            env.spec.id += f"-user_id={user_id}"
    return train_env, eval_env, test_env

In [ ]:
def sample_random_episode(env):
    episode_length, episode_return = 0, 0.0
    obs = env.reset()
    done = False
    while not done:
        action = env.action_space.sample()
        obs, reward, done, _ = env.step(action)
        episode_length += 1
        episode_return += reward
    return episode_length, episode_return

A célula abaixo implementa uma função que avalia uma política uniforme. Esse é um primeiro _baseline_ interessante para avaliarmos ao explorar novos problemas

In [ ]:
def eval_random_returns(env, episodes):
    episode_lengths, episode_returns = [], []

    for _ in trange(episodes):
        episode_length, episode_return = sample_random_episode(env)
        episode_lengths.append(episode_length)
        episode_returns.append(episode_return)

    episode_length_mean = np.mean(episode_lengths)
    episode_return_mean = np.mean(episode_returns)
    return episode_length_mean, episode_return_mean

In [ ]:
def test_random_agent(env):
    obs = env.reset()
    done = False
    episode_length, episode_return = 0, 0.0 
    while not done:
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        episode_length += 1
        episode_return += reward
        env.render()
        print()
    return info, episode_length, episode_return

### 1.1 Warm-up: único cliente, poucos produtos

Podemos passar `user_id` como argumento chave para construir um simulador das visitas de um único cliente. Isso facilita a inspeção do ambiente e serve como um caso base para testarmos estratégias simples.

In [ ]:
env1, _, test_env1 = make_shopping_cart_envs(max_products=5, user_id=54)

Os espaços de estado e ação tem tamanho em função do número de produtos (`max_products`)

In [ ]:
env1.observation_space, env1.action_space, env1.action_space.n

Podemos ver que as ações são vetores binários indicando quais produtos recomendar para o carrinho do usuário

In [ ]:
env1.action_space.sample()

Como "sanity check", podemos verificar se a ação amostrada está no espaço de ações.

In [ ]:
assert env1.action_space.sample() in env1.action_space

In [ ]:
sample_random_episode(env1)

Por fim, verificamos o desempenho médio da política uniforme em 100 episódios

In [ ]:
episodes = 100
eval_random_returns(env1, episodes)

In [ ]:
info, episode_length, episode_return = test_random_agent(test_env1)
pprint(info)
print(f"episode_length = {episode_length}, episode_return = {episode_return:.2f}")

### 1.2 Problema completo: vários clientes, mais produtos

In [ ]:
env2, _, test_env2 = make_shopping_cart_envs(max_products=100)

In [ ]:
episodes = 100
eval_random_returns(env2, episodes)

In [ ]:
info, episode_length, episode_return = test_random_agent(test_env2)
pprint(info)
print(f"episode_length = {episode_length}, episode_return = {episode_return:.2f}")

## 2. Escolhendo o algoritmo

- Dentre os algoritmos estudados em aula, qual o mais adequado? (A) DQN ou (B) SAC?
- É necessário alterar o algoritmo de alguma forma?


- Qual a especificação das redes (entrada e saída)? Que tipo de arquiterura (linear, linear + hand-engineered features, conv nets, MLP, ...)
- Comparação: redes independentes vs. features compartilhadas entre as predições de probabilidade de compra de cada item

## Q-Network

<img src="img/arch.png" alt="Qnet" style="width: 500px;"/>

Nossa arquitetura implementa a seguinte função $Q_{\phi} \colon S \mapsto \mathbb{R}^{2 |A|}$, ilustrada acima.

Note que decompomos o valor de uma ação (_slate_ de recomendações) em valores de recomendar ou não cada produto ao cliente.

O cálculo do valor do _slate_ final é a soma dos valores estimados de cada recomendação.
$$
Q_\phi(s, a) = \sum_k q_{\phi_k}(s, a_k)
$$

Através dessa implementação, estamos desconsiderando a influência do valor de um produto com o outro na hora da recomendação.

> É razoável essa fatoração? Formule argumentos a favor/contra essa decisão dependendo de sua posição

In [ ]:
class QNetwork(snt.Module):

    def __init__(self, observation_space, action_space, **config):
        super().__init__(name=config.get("name", "QNetwork"))

        self.observation_space = observation_space
        self.action_space = action_space

        self._torso = None
        if config.get("layers"):
            self._torso = snt.nets.MLP(
                config["layers"],
                activation=tf.nn.relu,
                activate_final=True,
                w_init=initializers.he_initializer(),
                name="MLP"
            ) 

        self._q_values = snt.Linear(2 * action_space.n, name="QValues")

    @tf.function
    def __call__(self, obs):
        """Calcula os Q-values de todas as ações para uma dada `obs`."""
        h = obs
        if self._torso:
            h = self._torso(h)
        return self._q_values(h)

    @tf.function
    def action_values(self, obs, actions):
        """Calcula os Q-values de uma única `action` específica para uma dada `obs`."""
        actions = tf.cast(actions, tf.int32)

        batch_size = tf.shape(obs)[0]
        actions_mask = tf.reshape(tf.one_hot(actions, depth=2), (batch_size, -1))
        tf.assert_equal(tf.shape(actions_mask), (batch_size, 2 * self.action_space.n))

        q_values = self(obs)
        tf.assert_equal(tf.shape(q_values), (batch_size, 2 * self.action_space.n))

        q_values = tf.reduce_sum(q_values * actions_mask, axis=-1)
        tf.assert_equal(tf.shape(q_values), (batch_size,))

        return q_values

    @tf.function
    def hard_update(self, other):
        """Copia os parâmetros da rede `other` para a rede do objeto."""
        for self_var, other_var in zip(self.trainable_variables, other.trainable_variables):
            self_var.assign(other_var)

In [ ]:
q_net = QNetwork(env1.observation_space, env1.action_space)

In [ ]:
_ = q_net(env1.observation_space.sample()[None])

In [ ]:
for variable in q_net.trainable_variables:
    print(variable.name, variable.shape, variable.dtype)

In [ ]:
batch_size = 128
obs = np.vstack([env1.observation_space.sample() for _ in range(batch_size)])
action = np.vstack([env1.action_space.sample() for _ in range(batch_size)])
action.shape, action.dtype

In [ ]:
q_values = q_net(obs)
assert q_values.shape == (batch_size, 2 * env1.action_space.n)

In [ ]:
q_values = q_net.action_values(obs, action)
assert q_values.shape == (batch_size,)

## Policy $\epsilon$-greedy

In [ ]:
class EpsilonGreedyPolicy:

    def __init__(self, q_net, start_val=1.0, end_val=0.01, start_step=1_000, end_step=10_000):
        self.q_net = q_net

        self._schedule = schedule.PiecewiseLinearSchedule((start_step, start_val), (end_step, end_val))

        self._step = tf.Variable(0., dtype=tf.float32, name="step")
        self._epsilon = tf.Variable(start_val, dtype=tf.float32, name="epsilon")

    def __call__(self, obs):
        """Retorna ação aleatória com probabilidade epsilon, c.c., retorna ação gulosa."""
        self._epsilon.assign(self._schedule(self._step))
        self._step.assign_add(1)
        
        d = tf.random.uniform(minval=0.0, maxval=1.0, shape=(1,))
        #print(f"epsilon = {float(self._epsilon):.3f}, d = {float(d):.3f}, ", end="")
        if self._epsilon > d:
            action = tf.random.uniform(minval=0.0, maxval=1.0, shape=(self.q_net.action_space.n,))
            action = action >= 0.5
            #print("~~ random ~~ :", end="")
        else:
            q_values = self.q_net(tf.expand_dims(obs, axis=0))
            q_values = tf.reshape(q_values, (-1, 2))
            action = tf.argmax(q_values, axis=-1)
            #print("!! GREEDY !! :", end="")

        action = tf.cast(action, self.q_net.action_space.dtype)
        #print(action)
        return action

In [ ]:
policy = EpsilonGreedyPolicy(q_net, start_val=1.0, end_val=0.1, start_step=10, end_step=50)

for i in range(100):
    print(f"i = {i:2d} => ", end="")
    action = policy(env1.observation_space.sample()).numpy()
    assert action.shape == (env1.action_space.n,)
    assert action in env1.action_space

### Double Q-Learning

In [ ]:
def make_double_q_learning_loss(q_net, target_q_net, gamma=0.99):
    """Recebe a rede online `q_net` e a rede `target_q_net` e devolve o loss function do Double Q-Learning."""

    @tf.function
    def _loss(batch):
        """Recebe um batch de experiências e devolve o valor da função objetivo para esse batch."""
        obs = batch["obs"]
        actions = batch["action"]
        rewards = batch["reward"]
        next_obs = batch["next_obs"]
        terminals = tf.cast(batch["terminal"], tf.float32)

        batch_size = tf.shape(obs)[0]

        # predictions
        q_values = q_net.action_values(obs, actions)

        # targets
        next_q_values = tf.reshape(q_net(next_obs), (batch_size, -1, 2))
        next_actions = tf.argmax(next_q_values, axis=-1, output_type=tf.int32)
        target_next_q_values = target_q_net.action_values(next_obs, next_actions)
        q_targets = tf.stop_gradient(rewards + (1 - terminals) * gamma * target_next_q_values)

        # loss = tf.reduce_mean((q_values - q_targets) ** 2)
        loss = tf.losses.huber(q_values, q_targets)
        return loss

    return _loss

In [ ]:
def make_update_fn(loss_fn, trainable_variables, learning_rate=1e-3):
    optimizer = snt.optimizers.Adam(learning_rate)

    @tf.function
    def _update_fn(batch):
        with tf.GradientTape(watch_accessed_variables=False) as tape:
            tape.watch(trainable_variables)
            loss = loss_fn(batch)

        grads = tape.gradient(loss, trainable_variables)
        optimizer.apply(grads, trainable_variables)

        grads_and_vars = {var.name: (grad, var) for grad, var in zip(grads, trainable_variables)}

        return loss, grads_and_vars

    return _update_fn

### DDQN - Double DQN

In [ ]:
class DDQN:

    def __init__(self, observation_space, action_space, config):
        self.observation_space = observation_space
        self.action_space = action_space

        self.config = config

        self.q_net = QNetwork(self.observation_space, self.action_space, **config["q_net"])
        self.target_q_net = QNetwork(self.observation_space, self.action_space, **config["q_net"])

        self.policy = EpsilonGreedyPolicy(self.q_net, **config["policy"])

        self._ckpt_dir = config["checkpoint_dir"]
        self._ckpt = tf.train.Checkpoint(q_net=self.q_net)
        self._ckpt_manager = tf.train.CheckpointManager(self._ckpt, directory=self._ckpt_dir, max_to_keep=1)

        self._step = tf.Variable(0, dtype=tf.int32, name="step")

    def build(self):
        """Cria as variáveis das redes online e target e sincroniza inicialmente."""
        input_spec = tf.TensorSpec(self.observation_space.shape, dtype=tf.float32)
        tf_utils.create_variables(self.q_net, input_spec)
        tf_utils.create_variables(self.target_q_net, input_spec)
        self.target_q_net.hard_update(self.q_net)

    def compile(self):
        """Compila a Double DQN loss junto com a DuelingQNetwork."""
        self.update_learner = make_update_fn(
            make_double_q_learning_loss(self.q_net, self.target_q_net, gamma=self.config["q_net"]["gamma"]),
            self.q_net.trainable_variables,
            learning_rate=self.config["q_net"]["learning_rate"]
        )

    def step(self, obs, training=True):
        """Escolhe a ação para a observação dada."""
        obs = tf.convert_to_tensor(obs, dtype=tf.float32)
        if training:
            action = self.policy(obs)
        else:
            q_values = self.q_net(tf.expand_dims(obs, axis=0))
            q_values = tf.reshape(q_values, (-1, 2))
            action = tf.argmax(q_values, axis=-1)
        return action.numpy()

    def learn(self, batch):
        """Recebe um batch de experiências, atualiza os parâmetros das redes, e devolve algumas métricas."""
        loss, grads_and_vars = self.update_learner(batch)

        # update target network
        self._step.assign_add(1)
        if self._step % self.config["q_net"]["target_update_freq"] == 0:
            self.target_q_net.hard_update(self.q_net)

        stats = {
            "loss": loss,
            "epsilon": self.policy._epsilon,
            "vars": {key: variable for key, (_, variable) in grads_and_vars.items()},
            "grads": {f"grad_{key}": grad for key, (grad, _) in grads_and_vars.items()},
        }

        return stats

    def save(self):
        """Salva o estado atual do agente (i.e., o valor dos parâmetros da rede online) nesse momento."""
        return self._ckpt_manager.save()

    def restore(self, save_path=None):
        """Carrega o último checkpoint salvo anteriormente no `save_path`."""
        if not save_path:
            save_path = self._ckpt_manager.latest_checkpoint
        return self._ckpt.restore(save_path)

### Protocolo de treinamento, avaliação e teste

In [ ]:
def train(
    agent,
    env,
    eval_env,
    replay,
    logger,
    total_timesteps=5000,
    learning_starts=500,
    learn_every=1,
    evaluation_freq=300
):  
    timesteps = 0
    episodes = 0
    episode_returns = deque(maxlen=20)

    best_episode_reward_mean = -np.inf
    
    with trange(total_timesteps, desc="training") as pbar:

        while timesteps < total_timesteps:
            episode_return = 0.0
            obs = env.reset()
            done = False

            train_stats, eval_stats = {}, {}

            for episode_length in itertools.count():

                # collect
                action = agent.step(obs, training=True)
                next_obs, reward, done, info = env.step(action)

                timesteps += 1
                episode_return += reward

                # add experience to replay buffer
                replay.add(obs, action, reward, done, next_obs)

                # training
                if timesteps >= learning_starts and timesteps % learn_every == 0:
                    batch = replay.sample()
                    train_stats = agent.learn(batch)

                # evaluation
                if timesteps % evaluation_freq == 0:
                    eval_stats = evaluate(agent, eval_env)

                    # logging
                    train_stats["episode_return_mean"] = np.mean(episode_returns)
                    logger.log(timesteps, train_stats, label="train") 
                    logger.log(timesteps, eval_stats, label="evaluation")

                if done:
                    break

                obs = next_obs

            episodes += 1
            episode_returns.append(episode_return)

            # logging
            stats = {
                "episodes": episodes,
                "episode_length": episode_length,
                "episode_return": episode_return,
            }
            logger.log(timesteps, stats, label="collect")
            logger.flush()

            pbar.update(episode_length)
            pbar.set_postfix(timesteps=timesteps, episodes=episodes, avg_returns=np.mean(episode_returns) if episode_returns else None)

    # final evaluation
    stats = evaluate(agent, eval_env)
    logger.log(timesteps, stats, label="evaluation")
    logger.flush()
    
    # checkpoint
    agent.save()

    return stats

In [ ]:
def evaluate(agent, env, episodes=20):
    episode_lengths, episode_returns = [], []

    for episode in range(episodes):
        episode_length, episode_return = 0, 0.0
        obs = env.reset()
        done = False
        while not done:
            action = agent.step(obs, training=False)
            obs, reward, done, _ = env.step(action)
            episode_length += 1
            episode_return += reward
        episode_lengths.append(episode_length)
        episode_returns.append(episode_return)

    return {
        "episode_return_mean": np.mean(episode_returns),
        "episode_return_min": np.min(episode_returns),
        "episode_return_max": np.max(episode_returns),
    } 

In [ ]:
def test(agent, env, episodes=3):
    stats = []

    for episode in range(episodes):
        episode_length, episode_return = 0, 0.0

        obs = env.reset()
        done = False

        while not done:
            action = agent.step(obs, training=False)
            obs, reward, done, info = env.step(action)
            episode_length += 1
            episode_return += reward
            env.render()
        
        stats.append({
            "info": info,
            "episode_length": episode_length,
            "episode_return": episode_return            
        })

    env.close()
    return stats

### Busca de hiperparâmetros

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs --reload_interval 10

In [ ]:
gz_file = pathlib.Path("instacart_online_grocery_shopping_2017_05_01.tar.gz")

# single-user
#env, eval_env, test_env = make_shopping_cart_envs(max_products=20, user_id=54)

# multiple-users
env, eval_env, test_env = make_shopping_cart_envs(max_products=20)

In [ ]:
total_timesteps = 5000

def run_experiment(config):
    timestamp = datetime.now().strftime("%Y-%m-%d-%H:%M")
    run_id = osp.join(f"ddqn-{env.spec.id}-{config['tags']}".lower(), timestamp, f"trial={config['trial_id']}")
    config["checkpoint_dir"] = f"ckpt/{run_id}"

    logger = logging.TFLogger(run_id, base_dir="logs")

    buffer = replay.ReplayBuffer(env.observation_space, env.action_space, max_size=total_timesteps, batch_size=config["q_net"]["batch_size"])
    buffer.build() 

    agent = DDQN(env.observation_space, env.action_space, config)
    agent.build()
    agent.compile()

    stats = train(agent, env, eval_env, buffer, logger, total_timesteps=total_timesteps)
    return stats

In [ ]:
def run_trials(configs, trials=5):
    results = defaultdict(list)

    for trial_id in range(trials):
        configs = [{**config, **{"trial_id": trial_id}} for config in configs]

        with mp.Pool(10) as p:
            run_stats = p.map(run_experiment, configs)

        for config, stats in zip(configs, run_stats):
            results[config["tags"]].append(stats)

    return results

### Experimento 1: taxa de aprendizado vs. batch size

In [ ]:
base_config = {
    "q_net": {
        "layers": [],
        "gamma": 0.99,
        "target_update_freq": 500,
        "learning_rate": 5e-3,
        "batch_size": 32,
    },
    "policy": {
        "start_val": 1.0,
        "end_val": 0.1,
        "start_step": 1_000,
        "end_step": 2_000
    },
}

params_names = ["bs", "lr"]
batch_sizes = [16, 64, 256]
learning_rates = [1e-3, 5e-3, 1e-2]

configs = []

for params in itertools.product(batch_sizes, learning_rates):
    batch_size, learning_rate = params

    config = copy.deepcopy(base_config)
    config["q_net"].update({"learning_rate": learning_rate, "batch_size": batch_size})

    tags = ",".join([f"{name}={value}" for name, value in zip(params_names, params)])
    config["tags"] = tags
    
    configs.append(config)

results = run_trials(configs, trials=3)

for config in configs:
    tags = config['tags']
    episode_return_means = [stats["episode_return_mean"] for stats  in results[tags]]
    print(f"{tags:20} : {episode_return_means}")

### Experimento 2: epsilon-greedy

In [ ]:
base_config = {
    "q_net": {
        "layers": [],
        "gamma": 0.99,
        "target_update_freq": 500,
        "learning_rate": 1e-3,
        "batch_size": 256
    },
    "policy": {
        "start_val": 1.0,
        "end_val": 0.1,
        "start_step": 1_000,
        "end_step": 2_000
    },
}

params_names = ["start", "end"]
start_steps = [100, 500, 1000]
end_steps = [500, 1000, 2000, 4000]

configs = []

for params in itertools.product(start_steps, end_steps):
    start_step, end_step = params
    if start_step >= end_step:
        continue

    config = copy.deepcopy(base_config)
    config["policy"].update({"start_step": start_step, "end_step": end_step})

    tags = ",".join([f"{name}={value}" for name, value in zip(params_names, params)])
    config["tags"] = tags

    configs.append(config)

results = run_trials(configs, trials=3)

for config in configs:
    tags = config['tags']
    episode_return_means = [stats["episode_return_mean"] for stats  in results[tags]]
    print(f"{tags:20} : {episode_return_means}")

### Experimento 3: arquitetura da Q-Network

In [ ]:
base_config = {
    "q_net": {
        "layers": [],
        "gamma": 0.99,
        "target_update_freq": 500,
        "learning_rate": 1e-3,
        "batch_size": 256
    },
    "policy": {
        "start_val": 1.0,
        "end_val": 0.1,
        "start_step": 500,
        "end_step": 2_000
    },
}

params_names = ["units"]
sizes = [8, 16, 32, 64, 256]

configs = []

for size in sizes:
    config = copy.deepcopy(base_config)
    config["q_net"].update({"layers": [size] * 2})
    config["tags"] = f"size={size}"
    configs.append(config)

results = run_trials(configs, trials=3)

for config in configs:
    tags = config['tags']
    episode_return_means = [stats["episode_return_mean"] for stats  in results[tags]]
    print(f"{tags:20} : {episode_return_means}")

## 3. Análise da solução

Qualitativamente como podemos avaliar o desempenho da solução obtida?

In [ ]:
config = {
    "q_net": {
        "layers": [256, 256],
        #layers": [],
        "gamma": 0.99,
        "target_update_freq": 500,
        "learning_rate": 1e-3,
        "batch_size": 256
    },
    "policy": {
        "start_val": 1.0,
        "end_val": 0.1,
        "start_step": 500,
        "end_step": 2_000
    },
    "checkpoint_dir": "ckpt/"
}

checkpoint_dir = "ckpt/ddqn-shoppingcart-v0-user_id=54-size=256/2021-02-04-18:27/trial=2/ckpt-1"
agent = DDQN(env.observation_space, env.action_space, config)
agent.build()
agent.restore(checkpoint_dir).assert_consumed()

stats = test(agent, test_env, episodes=1)
pprint(stats)

## 4. Decisões de modelagem - MDPs (caixa-branca)

**Simulador não-paramétrico:**
- Episódio: replay do log de transações do cliente (desenrolar a série histórica)
- Como dados históricos podem ser encapsulados como função de transição (replay do log de pedidos do cliente, ...)?

**Modelagem do MDP**
- Timestep (estágio de decisão) != noção de tempo => entre decisões intervalo de tempo diferentes
- env.reset(): múltiplos clientes => distribuição inicial $\rho$ => escolhe novo cliente para o episódio
- Função de transição: ação hoje não altera a tomada de decisão amanhã => o sistema não tem memória / stateless!
- Função de recompensa = tp - tn (feedback multi-dimensional vs. recompensa escalar) => incentivar mais produtos no slate com punição para produtos não escolhidos
- observabilidade parcial: timestamp da sessão + recency (compra)
- Espaço de ações tem tamanho fixo independente do cliente

**Sistemas de recomendação:**
- Tipicamente em sistemas de recomendação, as observações são compostas por no mínimo: 
    - perfil do cliente,
    - propensão de compra (cliente, produto), 
    - estatísticas de compras passadas do cliente, 
    - estatísticas de sucesso de cada produto (agregação das preferências dos usuários),
    - retorno acumulado como estatística de sucesso das recomendaçoes passadas para aquele cliente => sinal para aumentar ou diminuir a exploração

**Interpretação/análise da política:**
- política para 1 cliente => treinar demais em 1 cliente => overfitting/memorização => modelo "auto-regressivo" do histórico do cliente
- política para n clientes => modelo aprende política do cliente médio <=> tal produto vai bem em tal dia e horário, ou qual produto é mais vendável
- como customizar a política aprendida para cada cliente?

## 5. Considerações finais

**Modelagem do Problema:**
- Simulador => 1a opção, ponto de partida
- Efeitos de longo prazo importantes de se considear: por exemplo, errar muitas recomendações diminui o engajamento do cliente no futuro? recomendar muitos produtos pode ser detrimental limitações de UI? 
- Representação parcial do estado implica em necessidade de memória por parte do agente (e.g., RNNs)

**Algoritmos:**
- Nesse curso introdutório focamos em RL online com "simuladores" para apresentar os conceitos básicos de RL...
- No entanto, em aplicações reais (e-commerce, recomendadores, ...) => Off-line (batch) RL pode ser uma melhor solução!